In [31]:
# UNET parts

In [32]:
import torch
import torch.nn as nn


class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_op = nn.Sequential(

                                     nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                                     # Added to solve the issue of black prediction masks according to pytorch forum
                                     nn.BatchNorm2d(out_channels),
                                     nn.ReLU(inplace = True),
                                     nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
                                     # Added to solve the issue of black prediction masks according to pytorch forum
                                     nn.BatchNorm2d(out_channels),
                                     nn.ReLU(inplace=True)
        )

    def forward(self, x):
          return self.conv_op(x)

class DownSample(nn.Module):
    def __init__(self, in_channels, out_channels):
          super().__init__()
          self.conv = DoubleConv(in_channels, out_channels)
          self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
          down = self.conv(x)
          p = self.pool(down)
          return down, p

class UpSample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels//2, kernel_size = 2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        x = torch.cat([x1,x2], 1)
        return self.conv(x)

In [33]:
# Unet arch

In [34]:
class UNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.down_convolution_1 = DownSample(in_channels, 64)
        self.down_convolution_2 = DownSample(64, 128)
        self.down_convolution_3 = DownSample(128, 256)
        self.down_convolution_4 = DownSample(256, 512)

        self.bottle_neck = DoubleConv(512, 1024)

        self.up_convolution_1 = UpSample(1024, 512)
        self.up_convolution_2 = UpSample(512, 256)
        self.up_convolution_3 = UpSample(256, 128)
        self.up_convolution_4 = UpSample(128, 64)

        self.out = nn.Conv2d(in_channels =64, out_channels=num_classes, kernel_size=1)


    def forward(self, x):
        down_1, p1 = self.down_convolution_1(x)
        down_2, p2 = self.down_convolution_2(p1)
        down_3, p3 = self.down_convolution_3(p2)
        down_4, p4 = self.down_convolution_4(p3)

        b = self.bottle_neck(p4)

        up_1 = self.up_convolution_1(b, down_4)
        up_2 = self.up_convolution_2(up_1, down_3)
        up_3 = self.up_convolution_3(up_2, down_2)
        up_4 = self.up_convolution_4(up_3, down_1)

        out = self.out(up_4)
        return out




# Testing on random image

# if __name__ == "__main__":
#     double_conv = DoubleConv(256,256)
#     print(double_conv)

#     input_image = torch.rand((1,1,512,512))
#     model = UNet(1,2)
#     output = model(input_image)
#     print(output.size())

In [35]:
# Train loop

In [36]:
import os
from PIL import Image
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import cv2 as cv

root_path = "/home/readinggroup/Desktop/Image_proc_Noman/CSE465_project/dataset/segmentation_task/train"

class BrainTumorDataset(Dataset):
    def __init__(self, root_path):
        self.root_path = root_path
        self.images = sorted([root_path+"/images/"+i for i in os.listdir(root_path+"/images/")])
        self.masks = sorted([root_path+"/masks/"+i for i in os.listdir(root_path+"/masks/")])


        self.transform = transforms.Compose([
            transforms.ToTensor()
        ])

    def __getitem__(self, index):
        img = Image.open(self.images[index]).convert("L")
        mask = Image.open(self.masks[index]).convert("L")
        img = img.resize((512, 512), Image.BILINEAR)
        mask = mask.resize((512, 512), Image.NEAREST)
        return self.transform(img), self.transform(mask)


    def __len__(self):
        return len(self.images)



In [41]:
import torch
from torch import optim, nn
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

if __name__ == "__main__":
    # Hyper params for training
    LEARNING_RATE = 3e-4
    BATCH_SIZE = 8 # Keeping batch size low. Otherwise GPU will be out of memory.
    EPOCHS = 20
    arr_loss_per_epoch = []
    arr_val_loss_per_epoch = []
    best_val_loss = float("inf")

    DATA_PATH = "/home/readinggroup/Desktop/Image_proc_Noman/CSE465_project/dataset/segmentation_task/train"
    MODEL_SAVE_PATH = "./unet.pth"

    device = "cuda" if torch.cuda.is_available() else "cpu"
    train_dataset = BrainTumorDataset(DATA_PATH)


    generator = torch.Generator().manual_seed(42)
    train_dataset, val_dataset = random_split(train_dataset, [0.8, 0.2], generator=generator)

    train_dataloader = DataLoader(dataset=train_dataset,
                                batch_size=BATCH_SIZE,
                                shuffle=True)
    val_dataloader = DataLoader(dataset=val_dataset,
                                batch_size=BATCH_SIZE,
                                shuffle=True)


    model = UNet(in_channels=1, num_classes=1).to(device)
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.BCEWithLogitsLoss()

    for epoch in tqdm(range(EPOCHS)):
        model.train()
        train_running_loss = 0
        for idx, img_mask in enumerate(tqdm(train_dataloader)):
          # Training epoch
            
            img = img_mask[0].float().to(device)
            mask = img_mask[1].float().to(device)

            y_pred = model(img)
            optimizer.zero_grad()

            loss = criterion(y_pred, mask)
            train_running_loss += loss.item()

            loss.backward()
            optimizer.step()

        train_loss = train_running_loss / (idx + 1)


        # Validation epoch
        model.eval()
        val_running_loss = 0
        with torch.no_grad():
            for idx, img_mask in enumerate(tqdm(val_dataloader)):
                img = img_mask[0].float().to(device)
                mask = img_mask[1].float().to(device)

                y_pred = model(img)
                loss = criterion(y_pred, mask)

                val_running_loss += loss.item()

            val_loss = val_running_loss / (idx + 1)

        print("-"*30)
        print(f"Train Loss EPOCH {epoch+1}: {train_loss:.4f}")
        arr_loss_per_epoch.append(train_loss)
        print(f"Valid Loss EPOCH {epoch+1}: {val_loss:.4f}")
        arr_val_loss_per_epoch.append(val_loss)
        print("-"*30)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "best_unet.pth")

    print("The training loss per epoch:", arr_loss_per_epoch)
    print("The validation loss per epoch", arr_val_loss_per_epoch)

#     torch.save(model.state_dict(), MODEL_SAVE_PATH)

 12%|█████                                     | 95/787 [00:39<04:48,  2.40it/s]


 24%|██████████                               | 192/787 [01:19<04:08,  2.40it/s]


 37%|███████████████                          | 289/787 [02:00<03:27,  2.40it/s]


 49%|████████████████████                     | 386/787 [02:40<02:46,  2.40it/s]


 61%|█████████████████████████▏               | 483/787 [03:21<02:06,  2.40it/s]


...

 98%|████████████████████████████████████████▎| 775/787 [05:22<00:05,  2.40it/s]


100%|█████████████████████████████████████████| 787/787 [05:27<00:00,  2.40it/s]

 43%|█████████████████▉                        | 84/197 [00:12<00:17,  6.61it/s]


 92%|█████████████████████████████████████▋   | 181/197 [00:27<00:02,  6.64it/s]


100%|████████████████████████████████████████| 20/20 [1:59:16<00:00, 357.81s/it]

------------------------------
Train Loss EPOCH 20: 0.0064
Valid Loss EPOCH 20: 0.0141
------------------------------
The training loss per epoch: [0.15794177684729405, 0.04505650032183873, 0.02997129861019277, 0.023970957954912393, 0.020000919736945455, 0.017701113190066443, 0.01594618994984042, 0.01473629014836352, 0.013114126651811744, 0.012035481666073941, 0.011308040635854366, 0.009965992073813663, 0.010105061916080618, 0.0089627710458606, 0.008794168836355512, 0.007996792543509068, 0.007540964867903116, 0.007186190045013872, 0.0066831297463923215, 0.006394764861704964]
The validation loss per epoch [0.0668627655861644, 0.03598581187824156, 0.025570281171405376, 0.022474549852576353, 0.0385673965473995, 0.02040507449168088, 0.01713128800877384, 0.01688619180190079, 0.014633613581849536, 0.013449393529539484, 0.013568039607672673, 0.015671318718397678, 0.013577471621958586, 0.013473490783755626, 0.012639566945138998, 0.01376747354028302, 0.012799233719375608, 0.012225933648352772, 

In [42]:
 torch.save(model.state_dict(), MODEL_SAVE_PATH)

In [43]:
print(arr_val_loss_per_epoch)

[0.0668627655861644, 0.03598581187824156, 0.025570281171405376, 0.022474549852576353, 0.0385673965473995, 0.02040507449168088, 0.01713128800877384, 0.01688619180190079, 0.014633613581849536, 0.013449393529539484, 0.013568039607672673, 0.015671318718397678, 0.013577471621958586, 0.013473490783755626, 0.012639566945138998, 0.01376747354028302, 0.012799233719375608, 0.012225933648352772, 0.01269200862252962, 0.014131460090635846]
